In [1]:
# 🛠️ Install and Import Apache Airavata, the software which powers Cybershuttle
%pip install -qU "airavata-python-sdk[notebook]"
import airavata_jupyter_magic


%authenticate
%request_runtime hpc_gpu --file=cybershuttle.yml --walltime=60 --use=NeuroData25VC2:cloud
%wait_for_runtime hpc_gpu --live
%switch_runtime hpc_gpu

Note: you may need to restart the kernel to use updated packages.

Loaded airavata_jupyter_magic (2.1.4.post4) 
(current runtime = local)

  %authenticate                              -- Authenticate to access high-performance runtimes.
  %request_runtime <rt> [args]               -- Request a runtime named <rt> with configuration <args>.
                                                Call multiple times to request multiple runtimes.
  %restart_runtime <rt>                      -- Restart runtime <rt> if it hangs. This will clear all variables.
  %stop_runtime <rt>                         -- Stop runtime <rt> when no longer needed.
  %wait_for_runtime <rt>                     -- Wait for runtime <rt> to be ready.
  %switch_runtime <rt>                       -- Switch the active runtime to <rt>. All subsequent cells will run here.
  %%run_on <rt>                              -- Force a cell to always execute on <rt>, regardless of the active runtime.
  %stat_runtime <rt>               

Output()

Authenticated.

Requesting runtime=hpc_gpu...
[NeuroData25VC2:cloud, 60 Minutes, 1 Node(s), 4 CPU(s), 0 GPU(s), 8000 MB RAM, 0 MB VRAM]
* modules=[]
* libraries=['python=3.10', 'pip']
* pip=['inspect-ai', 'transformers', 'accelerate', 'tiktoken']
* mounts=[]
Requested runtime=hpc_gpu
Request successful: runtime=hpc_gpu


Output()

local:/tmp/connection_q0dz25hu.json --> hpc_gpu:connection_q0dz25hu.json... [200]
started proc_name=hpc_gpu_kernel on rt=hpc_gpu. pid=1823
forwarding ports=[15680, 15681, 15682, 15683, 15684]
hpc_gpu:15680 -> access via 18.118.140.230:10000
hpc_gpu:15681 -> access via 18.118.140.230:10001
hpc_gpu:15682 -> access via 18.118.140.230:10002
hpc_gpu:15683 -> access via 18.118.140.230:10003
hpc_gpu:15684 -> access via 18.118.140.230:10004
started ipykernel tunnels for hpc_gpu at 18.118.140.230
started ipykernel client for hpc_gpu
Remote Jupyter kernel launched and connected for runtime=hpc_gpu.
Switched to runtime=hpc_gpu.


<a href="https://colab.research.google.com/github/vash02/research-rep-agent/blob/main/agent_iterative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

executing cell on hpc_gpu...
waiting for cell to finish on hpc_gpu...
Wed May 14 03:47:29 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  GRID A100X-10C                 On  | 00000000:04:00.0 Off |                    0 |
| N/A   N/A    P0              N/A /  N/A |      1MiB / 10240MiB |      0%      Default |
|                                         |                      |             Disabled 

In [ ]:
%copy_data source=local:instructions.txt target=hpc_gpu:instructions.txt

%copy_data source=local:paper.md target=hpc_gpu:paper.md

%copy_data source=local:rubric.json target=hpc_gpu:rubric.json

%copy_data source=local:addendum.md target=hpc_gpu:addendum.md

In [2]:
# !pip install transformers accelerate
import os
os.environ["HF_TOKEN"] = {HF_TOKEN}

import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", trust_remote_code=True, torch_dtype=torch.bfloat16)


executing cell on hpc_gpu...
waiting for cell to finish on hpc_gpu...


/home/exouser/cybershuttle/scratch/envs/5a414a35/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Interrupt sent to remote kernel, waiting for response...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
Cell In[1], line 9
      7 from transformers import AutoTokenizer, AutoModelForCausalLM
      8 tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", trust_remote_code=True)
----> 9 model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", trust_remote_code=True, torch_dtype=torch.bfloat16)

File ~/cybershuttle/scratch/envs/5a414a35/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:571, in _BaseAutoModelClass.from_pretrained(cls, pretrained_model_name_or_path, *model_args, **kwargs)
    569     if model_class.config_class == config.sub_configs.get("text_config", None):
    570         config = config.get_text_config()
--> 571     return model_class.from_pretrained(
    572         pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs, **kwargs
   

In [4]:
from typing import List, cast
import tiktoken
from inspect_ai.model._chat_message import (
    ChatMessage,
    ChatMessageAssistant,
    ChatMessageSystem,
    ChatMessageTool,
    ChatMessageUser,
)

def handle_message_len(
    msg: ChatMessage,
    tokenizer,
    max_tokens: int,
) -> ChatMessage:
    def truncate_string(text: str, tokens: list[int], max_tokens: int) -> str:
        if len(tokens) > max_tokens:
            keep = max_tokens // 2
            first = tokenizer.decode(tokens[:keep])
            last = tokenizer.decode(tokens[-keep:])
            return first + "\n...[truncated]...\n" + last
        return text

    if isinstance(msg.content, str):
        tokens = tokenizer.encode(msg.content, disallowed_special=())
        msg.content = truncate_string(msg.content, tokens, max_tokens)
    return msg

def prune_messages(
    messages: List[ChatMessage], prune_individual: bool = True, max_context_tokens: int = 25000
) -> List[ChatMessage]:
    """Prune and truncate messages to prevent OOM and keep system prompt."""
    system_msgs = [m for m in messages if m.role == "system"]
    conversation = [m for m in messages if m.role != "system"]

    # Always keep the first user message (typically the task instruction)
    first_user = next((m for m in conversation if m.role == "user"), None)

    # Remove oldest ~50% of conversation (keep newer half)
    start = max(1, len(conversation) // 2)
    preserved = [first_user] if first_user else []
    preserved.extend(conversation[start:])

    # Tool-message consistency
    valid_msgs = []
    active_tool_ids = set()

    for m in preserved:
        if isinstance(m, ChatMessageAssistant):
            active_tool_ids = {tc.id for tc in (m.tool_calls or [])}
            valid_msgs.append(m)
        elif isinstance(m, ChatMessageTool) and getattr(m, "tool_call_id", None) in active_tool_ids:
            valid_msgs.append(m)
        elif isinstance(m, ChatMessageUser):
            active_tool_ids.clear()
            valid_msgs.append(m)

    if prune_individual:
        tokenizer = tiktoken.get_encoding("cl100k_base")
        MAX_PER_MSG = max_context_tokens // max(len(valid_msgs), 1)
        valid_msgs = [handle_message_len(m, tokenizer, MAX_PER_MSG) for m in valid_msgs]

    return cast(List[ChatMessage], system_msgs + valid_msgs)


In [7]:
import json
from pathlib import Path
from inspect_ai.model._chat_message import ChatMessage, ChatMessageUser, ChatMessageAssistant, ChatMessageSystem


class DiskBackedMessageHistory:
    def __init__(self, path="messages.jsonl", memory_window=3):
        self.path = Path(path)
        self.memory_window = memory_window
        self.messages_in_memory = []

        # Initialize file
        if self.path.exists():
            self.messages_in_memory = list(self._load_last_n(memory_window))
        else:
            self.path.write_text("")

    def append(self, message):
        self.messages_in_memory.append(message)
        with self.path.open("a") as f:
            f.write(json.dumps(message.model_dump()) + "\n")

        # Keep only last N messages in memory
        if len(self.messages_in_memory) > self.memory_window:
            self.messages_in_memory.pop(0)

    def get_recent(self):
        return self.messages_in_memory

    def __len__(self):
      with self.path.open() as f:
          return sum(1 for _ in f)

    def _deserialize_chat_message(self, raw: dict):
        role = raw.get("role")
        if role == "user":
            return ChatMessageUser(**raw)
        elif role == "assistant":
            return ChatMessageAssistant(**raw)
        elif role == "system":
            return ChatMessageSystem(**raw)
        else:
            raise ValueError(f"Unsupported role: {role}")

    def _load_last_n(self, n):
      with self.path.open() as f:
          lines = f.readlines()[-n:]
      return [self._deserialize_chat_message(json.loads(line)) for line in lines]



In [5]:
import torch
from inspect_ai.model import ModelOutput
from inspect_ai.model._chat_message import ChatMessageAssistant
from inspect_ai.model._model_output import ChatCompletionChoice

class LocalHFModelWrapper:
    def __init__(self, model, tokenizer, max_new_tokens=512):
        self.name = "local-llama3"
        self.model = model
        self.tokenizer = tokenizer
        self.max_new_tokens = max_new_tokens
        self.total_retry_time = 0
        self.api = None  # leave this None if not integrating with registry

    async def generate(self, input, tools=None, tool_choice=None, config=None):
        # Step 1: Construct prompt
        prompt = ""
        for msg in input:
            if msg.role == "system":
                prompt += f"[System]\n{msg.content}\n\n"
            elif msg.role == "user":
                prompt += f"[User]\n{msg.content}\n\n"
            elif msg.role == "assistant":
                prompt += f"[Assistant]\n{msg.content}\n\n"

        # Step 2: Tokenize safely and truncate if needed
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        # Step 3: Generate with model
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.max_new_tokens,
                pad_token_id=self.tokenizer.eos_token_id,
            )

        # Step 4: Decode response safely
        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Optional: Strip original prompt to get just the response (optional, depends on tokenizer)
        response = decoded[len(prompt):].strip()

        # Step 5: Return structured output
        return ModelOutput(
            model=self.name,
            choices=[ChatCompletionChoice(message=ChatMessageAssistant(content=response))]
        )


hf_model = LocalHFModelWrapper(model=model, tokenizer=tokenizer)

executing cell on hpc_gpu...
waiting for cell to finish on hpc_gpu...
cell finished on hpc_gpu.


In [7]:
import asyncio
import json
import gc
import torch
import re
from types import SimpleNamespace
from pathlib import Path
from typing import List
from inspect_ai.model._chat_message import (
    ChatMessageSystem, ChatMessageUser, ChatMessageAssistant, ChatMessageTool, ChatMessage
)
from inspect_ai.model._model_output import ChatCompletionChoice
from inspect_ai.model import ModelOutput
from inspect_ai.tool import Tool, tool
from inspect_ai.model._call_tools import call_tools


# --- Tool: Read file from path ---
@tool
def read_file() -> Tool:
    """Tool to read contents of a text file from disk."""
    async def execute(path: str) -> str:
        try:
            with open(path, "r") as f:
                return f.read()
        except Exception as e:
            return f"Error reading {path}: {str(e)}"
    return execute


from inspect_ai.tool import tool
from inspect_ai.tool._tool_call import ToolCallView, ToolCallContent

@tool(viewer=lambda tool_call: ToolCallView(
    call=ToolCallContent(
        title="Write File",
        format="markdown",
        content=f"📁 Writing to `{tool_call.arguments.get('path', '')}`:\n\n```python\n{tool_call.arguments.get('content', '')}\n```"
    )
))
def write_file():
    """Write content to a file at a given path."""

    async def execute(path: str, content: str) -> str:
        try:
            with open(path, "w") as f:
                f.write(content)
            return f"✅ File written successfully to: {path}"
        except Exception as e:
            return f"❌ Failed to write file: {str(e)}"

    return execute

from inspect_ai.tool import tool
import subprocess

@tool
def bash():
    """Execute a bash shell command and return its output."""
    async def execute(cmd: str) -> str:
        try:
            result = subprocess.run(
                cmd,
                shell=True,
                capture_output=True,
                text=True,
                timeout=30
            )
            output = result.stdout + ("\n" + result.stderr if result.stderr else "")
            return output.strip()
        except Exception as e:
            return f"Bash execution error: {str(e)}"
    return execute

@tool
def python():
    """Execute Python code and return its output."""
    async def execute(code: str) -> str:
        import io
        import contextlib

        stdout = io.StringIO()
        try:
            with contextlib.redirect_stdout(stdout):
                exec(code, {})
            return stdout.getvalue().strip()
        except Exception as e:
            return f"Python execution error: {str(e)}"
    return execute



# --- Token truncation helper ---
def handle_message_len(msg: ChatMessage, tokenizer, max_tokens: int) -> ChatMessage:
    def truncate(text: str, tokens: list[int], limit: int) -> str:
        if len(tokens) > limit:
            keep = limit // 2
            return tokenizer.decode(tokens[:keep]) + "\n...[truncated]...\n" + tokenizer.decode(tokens[-keep:])
        return text

    if isinstance(msg.content, str):
        tokens = tokenizer.encode(msg.content, disallowed_special=())
        msg.content = truncate(msg.content, tokens, max_tokens)
    return msg

# --- Pruning strategy ---
def prune_messages(messages: List[ChatMessage], max_context_tokens: int = 25000) -> List[ChatMessage]:
    from tiktoken import get_encoding
    tokenizer = get_encoding("cl100k_base")

    system_msgs = [m for m in messages if m.role == "system"]
    conversation = [m for m in messages if m.role != "system"]
    first_user = next((m for m in conversation if m.role == "user"), None)

    preserved = [first_user] if first_user else []
    preserved.extend(conversation[len(conversation)//2:])

    valid_msgs, active_tool_ids = [], set()
    for m in preserved:
        if m.role == "assistant":
            active_tool_ids = {tc.id for tc in (m.tool_calls or [])}
            valid_msgs.append(m)
        elif m.role == "tool" and getattr(m, "tool_call_id", None) in active_tool_ids:
            valid_msgs.append(m)
        elif m.role == "user":
            active_tool_ids.clear()
            valid_msgs.append(m)

    per_msg_limit = max_context_tokens // max(len(valid_msgs), 1)
    valid_msgs = [handle_message_len(m, tokenizer, per_msg_limit) for m in valid_msgs]

    return system_msgs + valid_msgs

# --- Extract code blocks ---
def extract_code_blocks(text, lang="python"):
    return re.findall(rf"```{lang}\n(.*?)\n```", text, re.DOTALL)

import torch
from inspect_ai.model import ModelOutput
from inspect_ai.model._chat_message import ChatMessageAssistant
from inspect_ai.model._model_output import ChatCompletionChoice

class LocalHFModelWrapper:
    def __init__(self, model, tokenizer, max_new_tokens=512):
        self.name = "local-llama3"
        self.model = model
        self.tokenizer = tokenizer
        self.max_new_tokens = max_new_tokens
        self.total_retry_time = 0
        self.api = None  # leave this None if not integrating with registry

    async def generate(self, input, tools=None, tool_choice=None, config=None):
        # Step 1: Construct prompt
        prompt = ""
        for msg in input:
            if msg.role == "system":
                prompt += f"[System]\n{msg.content}\n\n"
            elif msg.role == "user":
                prompt += f"[User]\n{msg.content}\n\n"
            elif msg.role == "assistant":
                prompt += f"[Assistant]\n{msg.content}\n\n"

        # Step 2: Tokenize safely and truncate if needed
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        # Step 3: Generate with model
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.max_new_tokens,
                pad_token_id=self.tokenizer.eos_token_id,
            )

        # Step 4: Decode response safely
        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Optional: Strip original prompt to get just the response (optional, depends on tokenizer)
        response = decoded[len(prompt):].strip()

        # Step 5: Return structured output
        return ModelOutput(
            model=self.name,
            choices=[ChatCompletionChoice(message=ChatMessageAssistant(content=response))]
        )


hf_model = LocalHFModelWrapper(model=model, tokenizer=tokenizer)
# --- Agent loop ---
async def agent_loop(state, model, max_messages=50):
    step = 0
    while not state.completed:
        if len(state.messages) >= max_messages:
            print("Reached message limit.")
            state.completed = True
            break

        pruned = prune_messages(state.messages)
        output = await model.generate(input=pruned[-20:], tools=state.tools)

        assistant_msg = output.choices[0].message
        print(f"\n─── Assistant turn {step} ─────────────────────────\n"
              f"{assistant_msg.content}\n")

        log_dir = Path("logs")
        log_dir.mkdir(exist_ok=True)
        with (log_dir / "assistant_steps.jsonl").open("a") as jf:
            jf.write(json.dumps({
                "step": step,
                "role": "assistant",
                "content": assistant_msg.content
            }) + "\n")
            # (optional) store the last user prompt that triggered this reply
            jf.write(json.dumps({
                "step": step,
                "role": "user_prompt",
                "content": pruned[-1].content if pruned else ""
            }) + "\n")
        state.messages.append(assistant_msg)

        # Code logging
        for code in extract_code_blocks(assistant_msg.content):
            print(f"\nStep {step} - Code:\n{code}\n")
            with open("generated_code_log.py", "a") as f:
                f.write(f"# === Step {step} ===\n{code}\n\n")

        # Tool call handling
        # Inside your agent_loop after model.generate(...)
        # Tool call handling (safe check)
        if assistant_msg.tool_calls:
            # Run tool calls in parallel
            async def handle_tool_call(tool_call):
                try:
                    if tool_call.function == "write_file":
                        result = await write_file().func(**tool_call.arguments)
                    elif tool_call.function == "read_file":
                        result = await read_file().func(**tool_call.arguments)
                    elif tool_call.function == "bash":
                        result = await bash().func(tool_call.arguments["cmd"])
                    elif tool_call.function == "python":
                        result = await python().func(tool_call.arguments["code"])
                    else:
                        result = f"Unknown tool: {tool_call.function}"
                    return ChatMessageTool(
                        tool_call_id=tool_call.id,
                        function=tool_call.function,
                        content=result
                    )
                except Exception as e:
                    return ChatMessageTool(
                        tool_call_id=tool_call.id,
                        function=tool_call.function,
                        content=f"Error during tool call: {str(e)}"
                    )

            tool_messages = await asyncio.gather(
                *[handle_tool_call(tc) for tc in assistant_msg.tool_calls]
            )
            state.messages.extend(tool_messages)
        else:
            state.messages.append(ChatMessageUser(content="No tool used. Continue generating more code."))


        torch.cuda.empty_cache()
        gc.collect()
        step += 1

    return state

tools = [read_file(), write_file(), bash(), python()]  # Add any other tools too


# --- Entry point ---
async def run():
    # 1. Load big resources once
    paper_md   = Path("paper.md").read_text()[:15000]   # first 15 k tokens
    rubric_js  = Path("rubric.json").read_text()
    addendum   = Path("addendum.md").read_text()[:8000]

    with open("instructions.txt") as f:
        instructions = f.read()

    # 2. Push them into the initial chat
    init_messages = [
        ChatMessageSystem(content=instructions),
        ChatMessageUser(content="Here is the rubric:\n```json\n" + rubric_js + "\n```"),
        ChatMessageUser(content="Here is the paper snippet:\n" + paper_md),
        ChatMessageUser(content="Addendum excerpt:\n" + addendum)
    ]

    state = SimpleNamespace(
        messages=init_messages,
        tools=tools,          # your read/write/bash/python list
        completed=False
    )

    await agent_loop(state, model=hf_model)

# Usage (Colab or notebook):
# await run()


executing cell on hpc_gpu...
waiting for cell to finish on hpc_gpu...


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.31s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


---------------------------------------------------------------------------
OutOfMemoryError                          Traceback (most recent call last)
Cell In[3], line 265
    256     state = SimpleNamespace(
    257         messages=[ChatMessageSystem(content=instructions)],
    258         tools=tools,
    259         completed=False
    260     )
    262     await agent_loop(state, model=hf_model)  # define hf_model elsewhere
--> 265 await run()

Cell In[3], line 262, in run()
    254     instructions = f.read()
    256 state = SimpleNamespace(
    257     messages=[ChatMessageSystem(content=instructions)],
    258     tools=tools,
    259     completed=False
    260 )
--> 262 await agent_loop(state, model=hf_model)

Cell In[3], line 205, in agent_loop(state, model, max_messages)
    202     break
    204 pruned = prune_messages(state.messages)
--> 205 output = await model.generate(input=pruned[-20:], tools=state.tools)
    207 assistant_msg = output.choices[0].message
    208 stat

In [ ]:
await run()